In [1]:
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import pandas as pd

This notebook contains all the functions required for accessing the Spotify Web API

In [1]:
def lookup_track(spotify, track_name):
    """
    Obtains the top Spotify track associated with the given track name.
    
    Input
        spotify; authenticated spotipy module
        track_name; string, the tracks name.
    Returns
        track; dictionary/json, data about the track
    """
    # If the data is not correctly stored return None.
    if track_name == None:
        return(None)
    
    results = spotify.search(q='track:' + track_name, type='track')
    items = results['tracks']['items']
    # If more than one spotify track is associated with that name return the top track.
    if len(items) > 0:
        track = items[0]
        return(track)
    else:
        # If the track is not found return none.
        print("track not found")
        return(None)
    
def lookup_artist(spotify, artist_name):
    """
    Obtains a the top artist associated with the artist's name.
    
    Input:
        spotify; authenticated spotipy module
        artist_name; string the artist's name
    
    Output:
        artist; dictionary/json, data about the artist
    """
    # If there data is not correctly stored return None.
    if artist_name == None:
        return(None)
    
    results = spotify.search(q='artist:' + str(artist_name), type='artist')  
    items = results['artists']['items']
    
    # if there is more than one artist associated to that artist name then return the top artist.
    if len(items) > 0:
        artist = items[0]['id']
        return(artist)
    else:
        # If there is no artist with that name return None.
        print("artist not found")
        return(None)    

In [4]:
def get_audio_features_names(spotify, track_names):
    """
    Returns each of the spotify audio features associated to each track in track names.
    
    Input
        spotify; authenticated spotipy module
        track_names; list of string, the list of track names to check.
    Output
        audio_features: Dataframe, A dataframe where each row represents a track
    """
    
    audio_features = []
    
    for track in track_names:
        #Make sure the track is not empty.
        if track != np.nan:
            try:
                # Can cause excepts so placed in a try catch.
                spoti_track = lookup_track(spotify, track)
            except:
                spoti_track = None
            if spoti_track != None:
                # Using the tracks ID look up the audio featurs for the track.
                audio_feature = spotify.audio_features(spoti_track['uri'])[0]
                if audio_feature != None:
                    audio_features.append(audio_feature)
    if audio_features != None:
        # Converts the audio features to a dataframe
        audio_features = pd.DataFrame(audio_features)
        audio_features = audio_features[['danceability','energy', 'loudness', 'acousticness', 'instrumentalness', 'speechiness', 'valence', 'tempo']]\

        return(audio_features)
    else:
        return(None)
    
def get_audio_features_ids(spotify, track_ids):
    """
    Returns each of the spotify audio features associated to each track in track ID's
    
    Input
        spotify; authenticated spotipy module
        track_ids; list of string, the list of track ids to check.
    Output
        audio_features: Dataframe, A dataframe where each row represents a track
    """
    audio_features = []
    # Since the track ID is already obtained there is no need to look up the tracks
    for idd in track_ids:
        # Get the top set of audio features associated with the given track ID.
        audio_feature = spotify.audio_features(idd)[0]
        if audio_feature != None:
            audio_features.append(audio_feature)
    if audio_features != None:
        # converts the audio features to a pandas Dataframe
        audio_features = pd.DataFrame(audio_features)
        audio_features = audio_features[['danceability','energy', 'loudness', 'acousticness', 'instrumentalness', 'speechiness', 'valence', 'tempo']]\

        return(audio_features)
    else:
        print("problem")
    

In [3]:
def get_playlist_tracks(spotify, playlist_uri):
    """
    Returns all the track IDs of tracks within a Spotify Playlist.
    
    Input
        spotify; authenticated spotipy module
        playlist_uri; string, the uri for a given playlist to lookup
    
    Output
        track_ids; list of string, each track id for a given playlist
    """
    # Looks up the playlist using spotipy
    results = spotify.playlist(playlist_uri)
    items = results['tracks']['items']
    track_ids = []
    for item in items:
        # Obtains each of the track ids.
        track_ids.append(item['track']['name'])
    return(track_ids)

In [5]:
def get_related_artist(spotify, artist_id):
    """
    Returns a list of artists considered similar based on Spotify community listening for a given artist
    
    Input:
        spotify; authenticated spotipy module
        artist_id; string, the artist's id
    
    Output:
        artist_names; list of strings, all artist names similar to the gien artist.
    """
    similar_artists = spotify.artist_related_artists(artist_id)
    artist_names = []
    for artist in (similar_artists['artists']):
        artist_names.append(artist['name'])
    return(artist_names)
    